# Try Pydantic

In [4]:
from pydantic import BaseModel, Field

class User(BaseModel):
    name: str = Field(description="The name of the user")
    age: int = Field(description="The age of the user")
    email: str = Field(description="The email of the user")

class Class(BaseModel):
    students: list[User] = Field(description="The students of the class")

In [5]:
foo_user = User(name="Joe", age=32, email="joe@example.com")

print(foo_user)

name='Joe' age=32 email='joe@example.com'


In [6]:
foo_class = Class(students=[User(name="Joe", age=32, email="joe@example.com")])
print(foo_class)

students=[User(name='Joe', age=32, email='joe@example.com')]


# Use pydantic to create OpenAI functions

In [7]:
class WeatherSearch(BaseModel):
    """Call with airport code to get the weather at the airport"""
    airport_code: str = Field(description="airport code to get the weather at the airport")

class ArtistSearch(BaseModel):
    """Call with artist name to get the artist's information"""
    artist_name: str = Field(description="artist name to get the artist's information")


In [8]:
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function

weather_function = convert_pydantic_to_openai_function(WeatherSearch)
weather_function

/var/folders/mw/s1_3z0zx20z4kvwtxs51stc80000gp/T/ipykernel_58253/84629652.py:3: LangChainDeprecationWarning: The function `_convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  weather_function = convert_pydantic_to_openai_function(WeatherSearch)


{'name': 'WeatherSearch',
 'description': 'Call with airport code to get the weather at the airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get the weather at the airport',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [9]:
from langchain_core.output_parsers import BaseOutputParser


class AnswerStrOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        # Split on </think> and take the content after it
        if '</think>' in text:
            return text.split('</think>', 1)[-1].strip()
        return text.strip()

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# Create ChatOpenAI instance
llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    model="deepseek/deepseek-r1-0528-qwen3-8b", 
    api_key="dummy",
    temperature=0)

# Bind the WeatherSearch function to the LLM
llm_with_tools = llm.bind_tools([WeatherSearch, ArtistSearch])

prompt = ChatPromptTemplate.from_messages([
    ('user', '{input}')
])

output_parser = AnswerStrOutputParser()

# Invoke the LLM with a weather query
chain = prompt | llm_with_tools 
response = chain.invoke({"input": "What's the weather like at LAX airport?"})

print(response.tool_calls)


[{'name': 'WeatherSearch', 'args': {'airport_code': 'LAX'}, 'id': '407533289', 'type': 'tool_call'}]


In [11]:
chain = prompt | llm_with_tools 
response = chain.invoke({"input": "What are three songs of the artist 'The Beatles'?"})

print(response.tool_calls)

[{'name': 'ArtistSearch', 'args': {'artist_name': 'The Beatles'}, 'id': '122762446', 'type': 'tool_call'}]


In [12]:
chain = prompt | llm_with_tools | output_parser
response = chain.invoke({"input": "Hi!"})

print(response)

Hello! How can I assist you today?


# Tagging

In [13]:
class Tagging(BaseModel):
    """Tag sentiment and language"""
    sentiment: str = Field(description="sentiment for e.g. positive, negative, neutral")
    language: str = Field(description="language code for e.g. en, it, es")

from langchain_core.utils.function_calling import convert_pydantic_to_openai_function
convert_pydantic_to_openai_function(Tagging)

{'name': 'Tagging',
 'description': 'Tag sentiment and language',
 'parameters': {'properties': {'sentiment': {'description': 'sentiment for e.g. positive, negative, neutral',
    'type': 'string'},
   'language': {'description': 'language code for e.g. en, it, es',
    'type': 'string'}},
  'required': ['sentiment', 'language'],
  'type': 'object'}}

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# Create ChatOpenAI instance
llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    model="deepseek/deepseek-r1-0528-qwen3-8b", 
    api_key="dummy",
    temperature=0)



llm_with_tools = llm.bind_tools([Tagging], tool_choice="required")
output_parser = JsonOutputToolsParser()

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Think carefully, and then tag the text as instructed.'),
    ('user', '{input}')
])

chain = prompt | llm_with_tools | output_parser
response = chain.invoke({"input": "The weather here is sunny and warm."})

response

[{'args': {'sentiment': 'positive', 'language': 'en'}, 'type': 'Tagging'},
 {'args': {'sentiment': 'positive', 'language': 'en'}, 'type': 'Tagging'},
 {'args': {'sentiment': 'positive', 'language': 'en'}, 'type': 'Tagging'}]

In [15]:
chain.invoke({"input": "I love LangChain!"})

[{'args': {'sentiment': 'positive', 'language': 'en'}, 'type': 'Tagging'},
 {'args': {'sentiment': 'positive', 'language': 'en'}, 'type': 'Tagging'}]

In [16]:
chain.invoke({"input": "non mi piace il cibo"})

[{'args': {'sentiment': 'negative', 'language': 'it'}, 'type': 'Tagging'},
 {'args': {'sentiment': 'negative', 'language': 'it'}, 'type': 'Tagging'}]

# Extraction

In [17]:
from typing import Optional, List
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a Person"""
    name: str = Field(description="name of the person")
    age: Optional[int] = Field(description="age of the person")

class Information(BaseModel):
    """Information to extract"""
    persons: List[Person] = Field(description="list of persons")

In [18]:
import logging
import httpx

# Enable httpx debug logging
logging.basicConfig(level=logging.WARN)
httpx_logger = logging.getLogger("httpx")
httpx_logger.setLevel(logging.WARN)

DEBUG = False

# Create a custom handler to see the HTTP calls clearly
class HTTPHandler(logging.Handler):
    def emit(self, record):
        if DEBUG and ('HTTP' in record.getMessage() or 'request' in record.getMessage().lower()):
            print(f"🌐 {record.getMessage()}")

http_handler = HTTPHandler()
httpx_logger.addHandler(http_handler)

In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser


extraction_llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    model="deepseek/deepseek-r1-0528-qwen3-8b", # "meta-llama-3.1-8b-instruct", 
    api_key="dummy",
    temperature=0)

extraction_llm_with_tools = extraction_llm.bind_tools([Person, Information])


prompt = ChatPromptTemplate.from_messages([
    ('system', 'Extract the information from the text and call the tools. DONOT guess the information.'),
    ('user', '{input}')
])

extraction_chain = prompt | extraction_llm_with_tools | JsonOutputToolsParser()
extraction_chain.invoke({"input": "John is 30 years old and lives in New York."})


[{'args': {'persons': [{'name': 'John', 'age': 30}]}, 'type': 'Information'},
 {'args': {'persons': [{'name': 'John', 'age': 30}]}, 'type': 'Information'},
 {'args': {'persons': [{'name': 'John', 'age': 30}]}, 'type': 'Information'}]

In [20]:
from langchain_core.output_parsers import JsonOutputKeyToolsParser, JsonOutputToolsParser

extraction_chain = prompt | extraction_llm_with_tools | JsonOutputToolsParser()
extraction_chain.invoke({"input": "John is 30 years old and he lives with his wife Martha"})

[]

# Extract authors form a post from web

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2025-05-01-thinking/")
docs = loader.load()
doc = docs[0]
doc.page_content[:100]

USER_AGENT environment variable not set, consider setting it to identify your requests.


"\n\n\n\n\n\nWhy We Think | Lil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n|\n\n\n\n\n\n"

In [1]:
from pydantic import BaseModel, Field

class Overview(BaseModel):
    """Overview of the document"""
    title: str = Field(description="Title of the document")
    summary: str = Field(description="Summary of the document")
    language: str = Field(description="Language of the document, e.g. en, it, es, etc.")
    keywords: list[str] = Field(description="keywords of the document")

class Paper(BaseModel):
    """Paper"""
    title: str = Field(description="Title of the paper")
    author: str = Field(description="Author of the paper")

class Info(BaseModel):
    """Information to extract"""
    papers: list[Paper] = Field(description="Papers in the document")

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.output_parsers import JsonOutputToolsParser, JsonOutputKeyToolsParser

doc_llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    model="meta-llama-3.1-8b-instruct", 
    api_key="dummy",
    temperature=0)

overiew_llm = doc_llm.bind_tools([Overview])

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Extract the overview of the document.'),
    ('user', '{input}')
])

doc_chain = prompt | overiew_llm | JsonOutputToolsParser() # | PydanticToolsParser(tools=[Overview])
doc_chain.invoke({"input": doc.page_content[:1000]})

[{'args': {'title': 'Why We Think | Lil',
   'summary': 'This document discusses various aspects of thinking, including motivation, analogy to psychology, computation as a resource, and latent variable modeling. It also explores thinking in tokens, branching and editing, parallel sampling, sequential revision, RL for better reasoning, external tool use, thinking faithfully, optimization pressure on CoT: good or bad?, thinking in continuous space, recurrent architecture, thinking tokens, thinking as latent variables, expectation-maximization, iterative learning, scaling laws for thinking time, and future directions.',
   'language': 'en',
   'keywords': '["thinking", "motivation", "psychology", "computation", "latent variable modeling", "tokens", "branching", "editing", "parallel sampling", "sequential revision", "RL", "external tool use", "faithfulness", "optimization pressure", "CoT", "continuous space", "recurrent architecture", "thinking tokens", "expectation-maximization", "iterati

In [19]:

info_llm = doc_llm.bind_tools([Info])

system_message = """
An article will be passed to you. Extract from it all papers.
DONOT extract the name of the article itself. If no papers are mentioned, return an empty list.
DONOT make up or guess any information. Only extract what's written in the article.
"""

prompt = ChatPromptTemplate.from_messages([
    ('system', system_message),
    ('human', '{input}')
])


doc_chain = prompt | info_llm | JsonOutputToolsParser()
doc_chain.invoke({"input": doc.page_content[:1000]})


[{'args': {'papers': '[{"title": "Motivation", "author": "Lilian Weng"}, {"title": "Analogy to Psychology", "author": "Lilian Weng"}, {"title": "Computation as a Resource", "author": "Lilian Weng"}, {"title": "Latent Variable Modeling", "author": "Lilian Weng"}, {"title": "Thinking in Tokens", "author": "Lilian Weng"}, {"title": "Branching and Editing", "author": "Lilian Weng"}, {"title": "Parallel Sampling", "author": "Lilian Weng"}, {"title": "Sequential Revision", "author": "Lilian Weng"}, {"title": "RL for Better Reasoning", "author": "Lilian Weng"}, {"title": "External Tool Use", "author": "Lilian Weng"}, {"title": "Thinking Faithfully", "author": "Lilian Weng"}, {"title": "Does the Model Tell What it Thinks Faithfully", "author": "Lilian Weng"}, {"title": "Optimization Pressure on CoT: Good or Bad?", "author": "Lilian Weng"}, {"title": "Thinking in Continuous Space", "author": "Lilian Weng"}, {"title": "Recurrent Architecture", "author": "Lilian Weng"}, {"title": "Thinking Tokens

In [5]:
from langchain_text_splitters.character import RecursiveCharacterTextSplitter

# Increase chunk size to handle larger text while staying within token limits
# With ~4 characters per token, 3000 characters ≈ 750 tokens (safe for 4096 limit)
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=200, chunk_size=3000)

splits = text_splitter.split_text(doc.page_content)
print(f"Number of splits: {len(splits)}")
print(f"Average split length: {sum(len(s) for s in splits) / len(splits):.0f} characters")
len(splits)

Number of splits: 24
Average split length: 2380 characters


24

In [26]:
from langchain_core.runnables import RunnableLambda

# create a list of dicts with each split
prep = RunnableLambda(lambda x: [{"input": split} for split in text_splitter.split_text(x)])
prep.invoke('hi')


[{'input': 'hi'}]

In [27]:
max ([len(input['input']) for input in prep.invoke(doc.page_content)])

2997

In [18]:
import json
json.loads("""
    [ { "papers" : "[{\"title\": \"Thinking, Fast and Slow\", \"author\": \"Daniel Kahneman\"}, {\"title\": \"Table of Contents\", \"author\": \"Lilian Weng\"}]" }]
""")


JSONDecodeError: Expecting ',' delimiter: line 2 column 24 (char 24)

In [22]:
from langchain_core.runnables import RunnableLambda

# create a list of dicts with each split
prep = RunnableLambda(lambda x: [{"input": split} for split in text_splitter.split_text(x)])
prep.invoke('hi')

# Create a runnable to flatten list of lists
# flatten_list = RunnableLambda(lambda x: sub_list['papers'] for sub_list in x)

whole_doc_chain = prep | doc_chain.map() 
response = whole_doc_chain.invoke(doc.page_content[:10000])


In [49]:
# Remove empty lists and flatten the results
papers_list = []
for response_record in response:
    for record in response_record:
        paper = record['args']['papers']
        print(paper, type(paper))

papers_list

[{"title": "Thinking, Fast and Slow", "author": "Daniel Kahneman"}, {"title": "Table of Contents", "author": "Lilian Weng"}, {"title": "Motivation", "author": "Lilian Weng"}, {"title": "Analogy to Psychology", "author": "Lilian Weng"}, {"title": "Computation as a Resource", "author": "Lilian Weng"}, {"title": "Latent Variable Modeling", "author": "Lilian Weng"}, {"title": "Thinking in Tokens", "author": "Lilian Weng"}, {"title": "Branching and Editing", "author": "Lilian Weng"}, {"title": "Parallel Sampling", "author": "Lilian Weng"}, {"title": "Sequential Revision", "author": "Lilian Weng"}, {"title": "RL for Better Reasoning", "author": "Lilian Weng"}, {"title": "External Tool Use", "author": "Lilian Weng"}, {"title": "Thinking Faithfully", "author": "Lilian Weng"}, {"title": "Does the Model Tell What it Thinks Faithfully", "author": "Lilian Weng"}, {"title": "Optimization Pressure on CoT: Good or Bad?", "author": "Lilian Weng"}, {"title": "Thinking in Continuous Space", "author": "L

[]